# Train and deploy with Cloud AI Platform

In [9]:
import os
import time

In [4]:
TRAINING_DATA_PATH = 'gs://jk-demo-datasets/gasdata/training.csv'
TESTING_DATA_PATH = 'gs://jk-demo-datasets/gasdata/testing.csv'
REGION = "us-central1"
ARTIFACT_BUCKET = 'gs://jk-demo-artifacts'
JOBDIR_BUCKET = 'gs://jk-demo-jobdir'

## Training with a Python script

In [5]:
TRAINING_APP_FOLDER = '../training_app/trainer'

os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)
!touch $TRAINING_APP_FOLDER/__init__.py

### Create a training script

In [6]:
%%writefile $TRAINING_APP_FOLDER/train.py

import logging
import os
import subprocess
import sys

import fire
import numpy as np
import pandas as pd

from sklearn.externals import joblib
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import Ridge
from sklearn.manifold import TSNE 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def train(job_dir, data_path, n_features_options, l2_reg_options):
    
  # Load data from GCS
  df_train = pd.read_csv(data_path)

  y = df_train.octane
  X = df_train.drop('octane', axis=1)
    
  pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dim', PCA()),
    ('regress', Ridge())
  ])

  param_grid = [
    {
      'reduce_dim__n_components': n_features_options,
      'regress__alpha': l2_reg_options
    }
  ]

  grid = GridSearchCV(pipeline, cv=10, n_jobs=None, param_grid=param_grid, scoring='neg_mean_squared_error', iid=False)
  
  grid.fit(X, y)

  logging.info("Best estimator: {}".format(grid.best_params_))
  logging.info("Best score: {}".format(grid.best_score_))
    
  # Retrain the best model on a full dataset
  best_estimator = grid.best_estimator_
  trained_pipeline = best_estimator.fit(X, y)

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=trained_pipeline, filename=model_filename)
  gcs_model_path = "{}/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  logging.info("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO)
  fire.Fire(train)

Writing ../training_app/trainer/train.py


### Configure dependencies

In [7]:
%%writefile $TRAINING_APP_FOLDER/../setup.py

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['fire', 'gcsfs']

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='My training application package.'
)

Writing ../training_app/trainer/../setup.py


## Submit a training job

In [10]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
SCALE_TIER = "BASIC"
MODULE_NAME = "trainer.train"
RUNTIME_VERSION = "1.14"
PYTHON_VERSION = "3.5"

N_FEATURES_OPTIONS="[2,4,6]"
L2_REG_OPTIONS="[0.1,0.2,0.3,0.5]"

In [11]:
!gcloud ai-platform jobs submit training $JOB_NAME \
--region $REGION \
--job-dir $JOBDIR_BUCKET/$JOB_NAME \
--package-path $TRAINING_APP_FOLDER \
--module-name $MODULE_NAME \
--scale-tier $SCALE_TIER \
--python-version $PYTHON_VERSION \
--runtime-version $RUNTIME_VERSION \
-- \
--data_path $TRAINING_DATA_PATH \
--n_features_options $N_FEATURES_OPTIONS \
--l2_reg_options $L2_REG_OPTIONS

Job [JOB_20190820_225215] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20190820_225215

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20190820_225215
jobId: JOB_20190820_225215
state: QUEUED


In [12]:
!gcloud ai-platform jobs describe $JOB_NAME
!gcloud ai-platform jobs stream-logs $JOB_NAME

createTime: '2019-08-20T22:52:21Z'
etag: 7qz7u7RC1E4=
jobId: JOB_20190820_225215
state: PREPARING
trainingInput:
  args:
  - --data_path
  - gs://jk-demo-datasets/gasdata/training.csv
  - --n_features_options
  - '[2,4,6]'
  - --l2_reg_options
  - '[0.1,0.2,0.3,0.5]'
  jobDir: gs://jk-demo-jobdir/JOB_20190820_225215
  packageUris:
  - gs://jk-demo-jobdir/JOB_20190820_225215/packages/3e43d633297470fd3897dfff3b5fc252e1d532fc508e64d3413fd1be90fc5774/trainer-0.1.tar.gz
  pythonModule: trainer.train
  pythonVersion: '3.5'
  region: us-central1
  runtimeVersion: '1.14'
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/JOB_20190820_225215?project=jk-demo1

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2FJOB_20190820_225215&project=jk-demo1
INFO	2019-08-20 22:52:21 +0000	service		Validating job requirements...
INFO	2019-08-20 22:52:21 +0000	service		Job creation request has been successfully validated.

### Copy the trained model to model repo

In [13]:
MODEL_NAME = "octane_regressor_r14"

!gsutil cp $JOBDIR_BUCKET/$JOB_NAME/model.joblib $ARTIFACT_BUCKET/models/$MODEL_NAME/model.joblib

Copying gs://jk-demo-jobdir/JOB_20190820_225215/model.joblib [Content-Type=application/octet-stream]...
/ [1 files][ 33.1 KiB/ 33.1 KiB]                                                
Operation completed over 1 objects/33.1 KiB.                                     


## Training with a custom container
### Create a training container image
#### Configure Cloud Build to use Kaniko

In [14]:
!gcloud config set builds/use_kaniko True

Updated property [builds/use_kaniko].


#### Create a Dockerfile

In [16]:
TRAINING_IMAGE_FOLDER = '../training_image'

os.makedirs(TRAINING_IMAGE_FOLDER, exist_ok=True)

In [17]:
%%writefile $TRAINING_IMAGE_FOLDER/Dockerfile

FROM gcr.io/jk-demo1/sklearn-cpu:latest
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Overwriting ../training_image/Dockerfile


#### Create a training script

In [18]:
%%writefile $TRAINING_IMAGE_FOLDER/train.py

import logging
import os
import subprocess
import sys
import joblib
import fire
import numpy as np
import pandas as pd

from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import Ridge
from sklearn.manifold import TSNE 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def train(job_dir, data_path, n_features_options, l2_reg_options):
    
  # Load data from GCS
  df_train = pd.read_csv(data_path)

  y = df_train.octane
  X = df_train.drop('octane', axis=1)
    
  pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dim', PCA()),
    ('regress', Ridge())
  ])

  param_grid = [
    {
      'reduce_dim__n_components': n_features_options,
      'regress__alpha': l2_reg_options
    }
  ]

  grid = GridSearchCV(pipeline, cv=10, n_jobs=None, param_grid=param_grid, scoring='neg_mean_squared_error', iid=False)
  
  grid.fit(X, y)

  logging.info("Best estimator: {}".format(grid.best_params_))
  logging.info("Best score: {}".format(grid.best_score_))
    
  # Retrain the best model on a full dataset
  best_estimator = grid.best_estimator_
  trained_pipeline = best_estimator.fit(X, y)

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=trained_pipeline, filename=model_filename)
  gcs_model_path = "{}/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  logging.info("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO)
  fire.Fire(train)

Overwriting ../training_image/train.py


#### Build the image

In [19]:
PROJECT_ID = !gcloud config list project --format "value(core.project)"
PROJECT_ID = PROJECT_ID[0]
IMAGE_REPO_NAME="octane-regression-training"
IMAGE_TAG="latest"
IMAGE_URI="gcr.io/{}/{}:{}".format(PROJECT_ID, IMAGE_REPO_NAME, IMAGE_TAG)

!gcloud builds submit --tag $IMAGE_URI $TRAINING_IMAGE_FOLDER

Creating temporary tarball archive of 2 file(s) totalling 1.9 KiB before compression.
Uploading tarball of [../training_image] to [gs://jk-demo1_cloudbuild/source/1566342876.51-e2fd899d389f443da947737c85c82426.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jk-demo1/builds/10fedaa2-42a3-4cd8-b062-b7dbb83f94a3].
Logs are available at [https://console.cloud.google.com/gcr/builds/10fedaa2-42a3-4cd8-b062-b7dbb83f94a3?project=826865698127].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "10fedaa2-42a3-4cd8-b062-b7dbb83f94a3"

FETCHSOURCE
Fetching storage object: gs://jk-demo1_cloudbuild/source/1566342876.51-e2fd899d389f443da947737c85c82426.tgz#1566342876976044
Copying gs://jk-demo1_cloudbuild/source/1566342876.51-e2fd899d389f443da947737c85c82426.tgz#1566342876976044...
/ [1 files][  1.0 KiB/  1.0 KiB]                                                
Operation completed over 1 objects/1.0 KiB.                                      
B

### Submit a training job

In [20]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
SCALE_TIER = "BASIC"

N_FEATURES_OPTIONS="[2,4,6]"
L2_REG_OPTIONS="[0.1,0.2,0.3,0.5]"

In [21]:
!gcloud ai-platform jobs submit training $JOB_NAME \
--region $REGION \
--job-dir $JOBDIR_BUCKET/$JOB_NAME \
--master-image-uri $IMAGE_URI \
--scale-tier $SCALE_TIER \
-- \
--data_path $TRAINING_DATA_PATH  \
--n_features_options $N_FEATURES_OPTIONS \
--l2_reg_options $L2_REG_OPTIONS

Job [JOB_20190820_231609] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20190820_231609

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20190820_231609
jobId: JOB_20190820_231609
state: QUEUED


In [22]:
!gcloud ai-platform jobs describe $JOB_NAME
!gcloud ai-platform jobs stream-logs $JOB_NAME

createTime: '2019-08-20T23:16:59Z'
etag: iE4ppY21-RE=
jobId: JOB_20190820_231609
state: PREPARING
trainingInput:
  args:
  - --data_path
  - gs://jk-demo-datasets/gasdata/training.csv
  - --n_features_options
  - '[2,4,6]'
  - --l2_reg_options
  - '[0.1,0.2,0.3,0.5]'
  jobDir: gs://jk-demo-jobdir/JOB_20190820_231609
  masterConfig:
    imageUri: gcr.io/jk-demo1/octane-regression-training:latest
  region: us-central1
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/JOB_20190820_231609?project=jk-demo1

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2FJOB_20190820_231609&project=jk-demo1
INFO	2019-08-20 23:16:58 +0000	service		Validating job requirements...
INFO	2019-08-20 23:16:59 +0000	service		Job creation request has been successfully validated.
INFO	2019-08-20 23:16:59 +0000	service		Job JOB_20190820_231609 is queued.
INFO	2019-08-20 23:16:59 +0000	service		Waiting for job to be provisioned.

### Copy the trained model to model repo

In [23]:
MODEL_NAME = "octane_regressor_container"

!gsutil cp $JOBDIR_BUCKET/$JOB_NAME/model.joblib $ARTIFACT_BUCKET/models/$MODEL_NAME/model.joblib

Copying gs://jk-demo-jobdir/JOB_20190820_231609/model.joblib [Content-Type=application/octet-stream]...
/ [1 files][ 33.1 KiB/ 33.1 KiB]                                                
Operation completed over 1 objects/33.1 KiB.                                     
